<a href="https://colab.research.google.com/github/A-NGJ/AudioExplorers2023/blob/main/1DCNN_withEnsembleLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2
from sklearn.preprocessing import normalize
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tqdm import tqdm
from scipy.stats import mode

def create_1d_cnn_leaky_model():
    model = tf.keras.Sequential([

        tf.keras.layers.Reshape((32, 96), input_shape=(32, 96, 1)),
        tf.keras.layers.Conv1D(32, 3, activation='relu'),
        # tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Conv1D(64, 3, activation='relu'),
        # tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Conv1D(128, 3, activation='relu'),
        # tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
        # tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(5, activation='softmax')
    ])

    lr_schedule = ExponentialDecay(initial_learning_rate=1e-3, decay_steps=10000, decay_rate=0.9)
    optimizer = Adam(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

X = np.load('training.npy')
y = np.load('training_labels.npy')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1875, random_state=1) # 0.1875 x 0.85 = 0.15

#X_train = np.load('X_train_augAJBig2.npy') Here load the augmented train data
#y_train = np.load('y_train_augAJBig2.npy')

y_valOrginal = y_val
y_testOriginal = y_test
# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=5)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=5)

# Reshape the data to have only one channel (for 1D convolutions)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

# Train 5 individual models
num_models = 2
# for one augmented model: 84%, 5 augmented models: 87.97%
models = [create_1d_cnn_leaky_model() for _ in range(num_models)]
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
for i, model in enumerate(models):
    print(f"Training model {i + 1}")
    model.summary()
    model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), verbose=0, callbacks=[early_stopping])
    number = str(i+1)
    model.save(number +'ensemble.h5')


# Ensemble predictions using voting
predictions = [model.predict(X_val) for model in models]
predictions = np.stack(predictions)
ensemble_predictions = np.argmax(np.mean(predictions, axis=0), axis=1)

# Evaluate the ensemble model
ensemble_accuracy = np.mean(ensemble_predictions == y_valOrginal)
print(f"Ensemble accuracyval: {ensemble_accuracy}")

# Ensemble predictions using voting
predictions = [model.predict(X_test) for model in models]
predictions = np.stack(predictions)
ensemble_predictions = np.argmax(np.mean(predictions, axis=0), axis=1)

# Evaluate the ensemble model
ensemble_accuracy = np.mean(ensemble_predictions == y_testOriginal)
print(f"Ensemble accuracyTEST: {ensemble_accuracy}")